# Interactive notebook used to prepare code parts for Azure ML

### Test MLFlow SDK 

In [2]:
mlflow_tracking_uri = "azureml://northeurope.api.azureml.ms/mlflow/v1.0/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourceGroups/d-azurelm/providers/Microsoft.MachineLearningServices/workspaces/aml-k8y9gydj"

In [3]:
import mlflow


In [4]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [17]:
from mlflow.tracking import MlflowClient

# experiment_name = 'lendingclub-demo-experiment'
# client = MlflowClient()

# exp = mlflow.get_experiment_by_name(experiment_name)
# print(exp)

# runs = mlflow.search_runs(experiment_names=experiment_name)
# print(runs[-1])

experiment_name = 'lendingclub-demo-experiment'
run_id = 'magenta_drawer_9k80sm4w8m'
filter_string = f"tags.mlflow.rootRunId='{run_id}'"
# runs2 = mlflow.search_runs(experiment_names=experiment_name, filter_string=filter_string)
runs = mlflow.search_runs(experiment_names=experiment_name, filter_string=filter_string, order_by=["metrics.val_accuracy"])
# runs = mlflow.search_runs(experiment_ids=None, search_all_experiments=True, filter_string=filter_string)

# print(finished_mlflow_run)
# metrics = finished_mlflow_run.data.metrics
# tags = finished_mlflow_run.data.tags
# params = finished_mlflow_run.data.params

# print(metrics,tags,params)

In [20]:
display(runs)

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [31]:
# Get best model
best_model = runs2[["run_id", "metrics.val_accuracy"]].sort_values(by="metrics.val_accuracy", ascending=False).head(1)
best_run_id = best_model["run_id"].iloc[0]
best_metric = best_model["metrics.val_accuracy"].iloc[0]

print(f"Best run id: {best_run_id}")
print(f"Best metric: {best_metric}")

Best run id: HD_e3040f1e-de19-4348-bcc7-d5cbd78d8236_0
Best metric: 0.8878837823867798


In [32]:
# Register model
model_name = "best_model"
model_path = f"runs:/{best_run_id}/model"
model = mlflow.register_model(model_uri=model_path, name=model_name)

Successfully registered model 'best_model'.
2022/12/16 13:23:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best_model, version 1
Created version '1' of model 'best_model'.
